In [1]:
from bs4 import BeautifulSoup as bs
import json
import requests
import pandas as pd

# from IPython.display import display, HTML
# from getpass import getpass

Для прекращения выполнения

In [2]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

Константы и переменные

In [83]:
HOSTSJ = 'https://www.superjob.ru'
hostSJ_geo = '%5Bc%5D%5B0%5D=1' #На всю страну Russia
page = 1
parsed_ht = bs()
vacancies = []
name_of_vac = input('Название вакансии:')

Название вакансии:python


Отправление запроса на сервер

In [88]:
def request_pageSJ(page):
    if page == 1:
        page = ''
    else:
        page = f'&page={page}'
    main_link = f'{HOSTSJ}/vacancy/search/?keywords={name_of_vac}&geo{hostSJ_geo}{page}'
    print(main_link)
    try:
        req = requests.get(main_link).text
    except Exception:
        print(f'При подключении {main_link} возникла ошибка.')
        raise StopExecution
    parsed_ht.global = bs(req,'lxml')
    print(f'Запрос {HOSTSJ}, страницы {page} без ошибок')

SyntaxError: invalid syntax (<ipython-input-88-2539502b379d>, line 13)

In [85]:
request_pageSJ(page)

https://www.superjob.ru/vacancy/search/?keywords=python&geo%5Bc%5D%5B0%5D=1
Запрос https://www.superjob.ru, страницы  без ошибок


In [86]:
while parsed_ht.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'}).text == 'дальше':
    parseSJ(parsed_ht)
    page += 1
    request_pageSJ(page)
else:
    page = 1
    parseSJ(parsed_ht)

AttributeError: 'NoneType' object has no attribute 'text'

In [69]:
df = pd.DataFrame(vacancies)
df

,site,position,salary_min,salary_max,salary,from,city,link
0,superjob.ru,Разработчик Python,0,0,По договорённости,Цифрасофт,Ижевск,https://www.superjob.ru/vakansii/razrabotchik-...
1,superjob.ru,Программист Python,0,0,По договорённости,Кадровое решение СПб,Санкт-Петербург,https://www.superjob.ru/vakansii/programmist-p...
2,superjob.ru,Программист Perl / Python,45000,45000,45 000 — 45 000 ₽,Дом.ru,Пермь,https://www.superjob.ru/vakansii/programmist-p...
3,superjob.ru,Аналитик python,0,0,По договорённости,Правительство Москвы,Москва,https://www.superjob.ru/vakansii/analitik-pyth...
4,superjob.ru,Программист на python,0,0,По договорённости,НТИМИ,Москва,https://www.superjob.ru/vakansii/programmist-n...
...,...,...,...,...,...,...,...,...
280,superjob.ru,Ведущий системный администратор Linux (Zabbix),0,0,По договорённости,Филиал ФКУ Налог-Сервис ФНС России по ЦОД в г....,Москва,https://www.superjob.ru/vakansii/veduschij-sis...
281,superjob.ru,Администратор баз данных PostgreSQL,0,0,150 000 ₽,Филиал ФКУ Налог-Сервис ФНС России по ЦОД в г....,Москва,https://www.superjob.ru/vakansii/administrator...
282,superjob.ru,Web-разработчик React,0,0,По договорённости,КРИТ,Ижевск,https://www.superjob.ru/vakansii/web-razrabotc...
283,superjob.ru,Тестировщик программного обеспечения,0,0,По договорённости,Правительство Москвы,Москва,https://www.superjob.ru/vakansii/testirovschik...


In [42]:
def salary_eject(salary):
    if not salary or 'договорённости' in salary:
        return 0, 0
    salary = ''.join(salary.split())
    if 'от' in salary:
        salary = salary.replace('от', '').replace('₽', '')
        return int(salary), 0
    if 'до' in salary:
        salary = salary.replace('до', '').replace('₽', '')
        return 0, int(salary)
    if '—' in salary:
        salary = salary.replace('₽', '')
        salary = salary.split('—')
        return int(salary[0]), int(salary[1])
    return (0, 0)

In [64]:
def parseSJ(parsed_html):
    div_block = parsed_html.find_all('div', {'class':'_3syPg _3P0J7 _9_FPy'})
    for div in div_block:
        vac_data={}
        try:
            a_all = div.find_all('a')
            # position
            name_div = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
            # salary
            summ_span = div.find('span', {'class': 'f-test-text-company-item-salary'}).text
            # link
            href_a = a_all[0]['href']
            href_link = HOSTSJ + href_a
            # or from who
            comp = a_all[1].getText()
            # data
            time_div = div.find('span', {'class': '_3mfro _9fXTd _2JVkc _3e53o _3Ll36'})
            # city
            town_div = time_div.findNextSiblings()[0].text
            vac_data['site'] = 'superjob.ru'
            vac_data['position'] = name_div
            salary = salary_eject(summ_span)
            str_salary = summ_span.replace('\xa0',' ')
            #             sum_min, sum_max = salary_sj(str_salary)
            vac_data['salary_min'] = salary[0]
            vac_data['salary_max'] = salary[1]
            vac_data['salary'] = str_salary
            vac_data['from'] = comp
            vac_data['city'] = town_div
            #vac_data['time'] = time_div.getText()
            vac_data['link'] = href_link
            vacancies.append(vac_data)
        except Exception  as e:
            print(e)

list index out of range


,site,position,salary_min,salary_max,salary,from,city,link
0,superjob.ru,Разработчик Python,0,0,По договорённости,Цифрасофт,Ижевск,https://www.superjob.ru/vakansii/razrabotchik-...
1,superjob.ru,Программист Python,0,0,По договорённости,Кадровое решение СПб,Санкт-Петербург,https://www.superjob.ru/vakansii/programmist-p...
2,superjob.ru,Программист Perl / Python,45000,45000,45 000 — 45 000 ₽,Дом.ru,Пермь,https://www.superjob.ru/vakansii/programmist-p...
3,superjob.ru,Аналитик python,0,0,По договорённости,Правительство Москвы,Москва,https://www.superjob.ru/vakansii/analitik-pyth...
4,superjob.ru,Программист на python,0,0,По договорённости,НТИМИ,Москва,https://www.superjob.ru/vakansii/programmist-n...
5,superjob.ru,Backend разработчик (Python / Django),80000,190000,80 000 — 190 000 ₽,СтройУчетСервис,Москва,https://www.superjob.ru/vakansii/backend-razra...
6,superjob.ru,Тестировщик НТ,0,0,По договорённости,Сбербанк России,Москва,https://www.superjob.ru/vakansii/testirovschik...
7,superjob.ru,Разработчик Витрин Данных,0,0,По договорённости,Сбербанк России,Москва,https://www.superjob.ru/vakansii/razrabotchik-...
8,superjob.ru,Разработчик Oracle PL/SQL,0,0,По договорённости,РЕСО-Гарантия,"Москва,Нахимовский проспект",https://www.superjob.ru/vakansii/razrabotchik-...
9,superjob.ru,Администратор тестовых сред (ведущий инженер),0,0,По договорённости,Сбербанк России,Санкт-Петербург,https://www.superjob.ru/vakansii/administrator...
